<h1>
<center>
Assignment 5: PCA, Naive Bayes, Clustering, and Deep Learning
</center>
</h1>
<center>
CS 4262/5262 - Foundations of Machine Learning<br>
Vanderbilt University, Spring 2023<br>
Due: Check Brightspace
</center>
<hr>
<br>This assignment will focus on: (1) PCA, (2) Clustering, and (3) deep learning. In addition to programming tasks, there are short-answer questions throughout the notebook. 

Contact: Quan Liu quan.liu@vanderbilt.edu for any clarifying questions.


### Please enter your name:  




--- 
## Part 1: Eigencats


In this part, you will be working with a dataset of cat images. You will apply principal component analysis (PCA) to decompose the data into "eigencats", and use the eigencat basis to form low-dimensional approximations of the original data.

- Load the cat dataset from the file "cats.csv" (should be included in the git repo; here is the original source: https://github.com/bioramble/pca). This dataset consists of a 4096 x 80 matrix of "pre-flattened" 64 x 64 images of cats (i.e., each of eighty 64 x 64 image has been flattened into a vector of length 4096).
- A function for loading the dataset has been provided below, so you just need to call it.


In [ ]:
# code for loading dataset into matrix
# returns a matrix where each of the 80 rows represents a cat image 
# and the 4096 columns represent features (pixels) of the images
def load_cat_file(file_path):
    cat_matrix = np.zeros((80,4096))
    with open(file_path) as f:
        cat_reader = csv.reader(f, delimiter=',')
        for i, line in enumerate(cat_reader):
            pixels = [int(p) for p in line]
            cat_matrix[:,i] = pixels
    return cat_matrix
            
cat_matrix = load_cat_file('cat.csv')

- The next block of code provides a function for visualizing a single cat image, given a flattened image as input.
   


In [ ]:
# function for visualizing a cat image
def vis_cat_image(cat_vec, axis=None):
    cat_img = np.reshape(cat_vec, (64,64)).T 
    if axis is None:
        plt.imshow(cat_img, interpolation='nearest')
        plt.show()
    else: 
        axis.imshow(cat_img, interpolation='nearest')

 * To familiarize yourself with the data, use the function above to display a single cat image of your choosing.
   

* Next, display the average over all 80 cat images:

You will next perform PCA on the cat dataset to extract a set of "eigencats".
- Recall that it is important to perform feature scaling prior to PCA. First, please mean-center and standardize over features in the cat dataset.
- Next, calculate the eigenvectors and eigenvalues of the covariance matrix. **Do not use PCA from scikit-learn**, but you can use numpy functions. Please be sure to read and understand the documentation of those functions. 
- Display (as images) the top 16 "eigencats" (i.e., the 16 eigenvectors corresponding to the largest 16 eigenvalues). You can use the function `vis_cat_image` above. Place a title on each image that indicates which of the eigencats is displayed, where 'eigencat 1' corresponds to the largest eigenvalue and 'eigencat 16' corresponds to the 16th-largest eigenvalue.
- Display a plot of the eigenvalues, ordered from largest to smallest. Include a title and label the axes.

In [ ]:
import numpy as np
from scipy import linalg as LA


Now, you will use your eigencats to form a low-dimensional approximation of the data. Each original (scaled) cat image can be approximated by a weighted sum of eigencats, where the weight associated with the $i^{th}$ eigencat corresponds to the projection of the original cat image onto the $i^{th}$ eigencat. Since the eigencats have norm = 1 (you can verify this!), the projection of an image onto each eigencat is simply the inner product between the image and the eigencat.

- Select an image of your choice from the original (scaled) dataset. 
   * Calculate the projection of this image onto the set of all principal components. This will result in a vector of length 4096, whose entries correspond to weights associated with each eigencat.
   * Reconstruct the image from the set of *all* principal components. Display the resulting image.
   * Reconstruct the image using only 1,2,8,16, and 32 principal components. Display all resulting images.
- Repeat the above steps using another randomly selected image.


**Question 1** Write down some of your observations about the appearance of the eigencats.

**Question 2** What happens to the reconstructed image as you include more and more principal components? 


--- 
## Part 2: Clustering


In this part, we will perform clustering on the cat images using k-means. Since the images are quite high-dimensional (4096 dimensions), we will use PCA to project the data into a lower-dimensional space prior to clustering. 

**Question 3**  Based on your analysis in Part 1, how many principal components are needed to explain 90% of the variance in the data?


- Project each cat image onto the set of $L$ eigencats that capture 90% variance in the data. Now, each image is represented by $L$ coordinates (i.e., its projection onto $L$ eigencats) instead of 4096.
- Perform k-means clustering on the resulting vectors, using K=4. You may use functions from scikit-learn.
- This results in 4 cluster centroids. Project those cluster centroids back into the original (4096-dimensional) space, and display each of those 4 centroid cats using the function `viz_cat_image` above.

--- 
## Part 3: Deep Learning for Sentiment Analysis

 

In this part, you will be using Keras/PyTorch to build a fully-connected, feed-forward neural network for sentiment analysis. The [dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) you will be working with contains 50,000 movie reviews from IMDB and the corresponding labels, where the label indicates whether the review was positive or negative. 

If you are using Keras, make sure you install the tensorflow at first since keras is just a wrapper API package for tensorflow

In [ ]:
from keras.datasets import imdb 

# todo: add more keras/pytorch imports as needed 

- To begin, run the code block below to load the training and test sets of IMDB data from Keras/PyTorch. Each review (sample) is represented as a sequence of integers, one integer for each word in that review, where the integer assigned to a given word corresponds to the index of that word in the vocabulary. Note that below, we will turn these samples into fixed-length binary vectors. 

In [ ]:
# fetch dataset from keras (already has a 50/50 train/test split)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000, seed=113)

# concatenate the data (N = 50,000)
X = np.concatenate((x_train, x_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

# transform X such that every input is the same length 
## 1 if input x contains the word, 0 otherwise
## so, we are ignoring word frequency 
X_vec = np.zeros((50000,10000))
for i, x in enumerate(X):
    X_vec[i, x] = 1

# 80/20 train/test split
x_train = X_vec[:40000]
y_train = y[:40000]
x_test = X_vec[40000:]
y_test = y[40000:]

- Using the Keras/PyTorch framework, build a fully-connected, feed-forward neural network for the IMDB dataset. This network should take in the binary vector representations of movie reviews and predict the positive/negative sentiment of the reviews. 
    * Take advantage of Keras’ online documentation. Additional supplemental materials by the TA may be referenced, but should not be used as a template for your homework.
- Hint: use binary cross-entropy as your loss function. 
- **Submit a neural network that reaches >= 85% accuracy on the test set.**

**Question 4** Specify the optimization method and activation function(s) that you used.

**Question 5** Provide a brief reflection on the process by which you tweaked your network to improve its performance.


---
## Part 4: Submission 

Please upload a clean version of your work to Brightspace by the deadline. Below, please acknowledge your collaborators as well as any resources/references (beyond Keras and guides to Python syntax) that you have used in this assignment: